In [1]:
import numpy as np
import librosa as lba
import soundfile as sf
from random import random

In [2]:
stimuli_folder = '../stimuli/'

sr = 44100
tone_duration = 250

rise_duration = 10
rise_length = int(rise_duration * sr / 1000)

perc_duration = 240
perc_length = int(perc_duration * sr / 1000)

fade_duration = 10
fade_length = int(fade_duration * sr / 1000)

# Determine frequencies of tones up to 100 cents above and below A4 by multiplying or dividing by 2^(n/1200), where n is the number of cents
tones = {'A4-%i' % n: 440 / 2**(n/1200) for n in range(1, 101, 1)} | {'A4': 440} | {'A4+%i' % n: 440 * 2**(n/1200) for n in range(1, 101, 1)}

# Generate Tones

In [3]:
for pitch in tones:

    f = tones[pitch]
    
    # Generate fundamental frequency (co)sine wave
    tone = lba.tone(f, sr=sr, duration=tone_duration / 1000)

    # Add first three harmonics (up to 2 octaves above) with slope of -3 db/half amplitude per octave
    for i in range(3):
        phase = random() * 2 * np.pi
        tone += lba.tone(f * (i + 2), sr=sr, duration=tone_duration / 1000, phi=phase) / (i + 2)

    # Apply exponential fade to create percussive envelope
    tone[-perc_length:] *= np.geomspace(1, .01, perc_length)
    # Apply short linear fade to ending so that amplitude fades to 0
    tone[-fade_length:] *= np.linspace(1, 0, fade_length)
    # Apply sharp linear rise to start of tone
    tone[:rise_length] *= np.linspace(0, 1, rise_length)

    # Rescale waveform to range [-1, 1] to prevent clipping
    tone /= np.abs(tone).max()
    
    # Save tone
    sf.write('../stimuli/tones/tone%s.wav' % pitch, tone, 44100)